In [1]:
import pandas as pd

df = pd.read_csv("./SbD4Nano_landscape_relations - Blad2.csv")

In [2]:
df.head()

,Node_name,Node,IRI,Gracious,Node1,causes,Node2,Cito,Reference
0,N1,core composition,http://purl.bioontology.org/ontology/npo#NPO_1808,gracious:fa7326c4-966e-2b43-ae02-228eeeaf8de6,N11,correlates,N10,containsAssertionFrom,10.1039/d0nr00868k
1,N2,coating/surface chemistry,http://purl.bioontology.org/ontology/npo#NPO_1823,gracious:0f790f0f-2b26-d346-8756-4ea6181af3c4,N12,correlates,N10,containsAssertionFrom,https://dr.lib.iastate.edu/server/api/core/bit...
2,N3,dimensionality (shape),NaN,NaN,N9,correlates,N10,containsAssertionFrom,https://dr.lib.iastate.edu/server/api/core/bit...
3,N4,polydispersity,http://purl.bioontology.org/ontology/npo#NPO_274,http://www.bigcat.unimaas.nl/sbd4nano/gracious...,N2,correlates,N10,containsAssertionFrom,https://dr.lib.iastate.edu/server/api/core/bit...
4,N5,diameter,NaN,NaN,N10,correlates,N14,containsAssertionFrom,10.1039/d0nr00868k


In [3]:
df.columns

Index(['Node_name', 'Node', 'IRI', 'Gracious', 'Node1', 'causes', 'Node2',
       'Cito', 'Reference'],
      dtype='object')

In [4]:
df.IRI. unique()

array(['http://purl.bioontology.org/ontology/npo#NPO_1808',
       'http://purl.bioontology.org/ontology/npo#NPO_1823', nan,
       'http://purl.bioontology.org/ontology/npo#NPO_274',
       'http://purl.bioontology.org/ontology/npo#NPO_895',
       'http://purl.bioontology.org/ontology/npo#NPO_1967',
       'http://purl.bioontology.org/ontology/npo#NPO_1853',
       'http://purl.bioontology.org/ontology/npo#NPO_1955',
       'http://purl.org/obo/owl/PATO#PATO_0000117'], dtype=object)

In [5]:
node_df = df[['Node_name', 'Node', 'IRI', 'Gracious']]
network_df = df[['Node1', 'causes', 'Node2', 'Cito', 'Reference']]

In [6]:
node_df.Node[0]

'core composition'

In [7]:
testlist=["N1", "N6"]

for node in testlist:
    print(node_df[node_df.Node_name == node])

  Node_name              Node  \
0        N1  core composition   

                                                 IRI  \
0  http://purl.bioontology.org/ontology/npo#NPO_1808   

                                        Gracious  
0  gracious:fa7326c4-966e-2b43-ae02-228eeeaf8de6  
  Node_name    Node  IRI Gracious
5        N6  length  NaN      NaN


In [8]:
import re
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import DC, RDFS, FOAF, DCTERMS, VOID, RDF, XSD, OWL
# CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
# PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
# VOID, XMLNS, XSD

sbd = Namespace("https://www.sbd4nano.eu/rdf/#")
sbdbel = Namespace("https://www.sbd4nano.eu/bel/#")
ECO = Namespace("https://evidenceontology.org/#")
kb = Namespace("https://h2020-sbd4nano.github.io/sbd-data-landscape/")
enm = Namespace("http://purl.enanomapper.org/onto/")
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT")
npo = Namespace("http://purl.bioontology.org/ontology/npo#")
pato = Namespace("http://purl.org/obo/owl/PATO#")
cito = Namespace("http://purl.org/spar/cito/")

In [32]:
g = Graph()
g.bind("dc", DC)
g.bind("rdfs", RDFS)
g.bind("foaf", FOAF)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("void", VOID)
g.bind("rdf", RDF)
g.bind("xsd", XSD)
g.bind("eco", ECO)
g.bind("sbd", sbd)
g.bind("sbdbel", sbdbel)
g.bind("kb", kb)
g.bind("enm", enm)
g.bind("ncit", ncit)
g.bind("npo", npo)
g.bind("pato", pato)
g.bind("cito", cito)

source = URIRef("https://docs.google.com/spreadsheets/d/18n37M72zQJgfsELIGRj6llEZ44Ao48qGoXWXyPazRD0/edit#gid=760845260")

for i, row in enumerate(network_df.index):
    print(i)
    selected_row = network_df.loc[row]
    relation = URIRef("https://h2020-sbd4nano.github.io/sbd-data-landscape/" + "CausalAssertion" + "_" + str(i))
    cause = BNode()
    cause_node = node_df.loc[node_df.Node_name == str(selected_row.Node1).strip()].index
    outcome = BNode()
    outcome_node = node_df[node_df.Node_name == str(selected_row.Node2).strip()].index
    g.add((relation, RDF.type, URIRef(sbd + "resource")))
    g.add((relation, RDF.type, URIRef(sbd + "causalAssertion")))
    g.add((relation, DC.source, source))
    g.add((relation, cito.containsAssertionFrom, URIRef(str(selected_row.Reference))))
    
    #cause
    g.add((relation, URIRef(sbdbel + "cause"), cause))
    g.add((cause, DCTERMS.label, Literal(node_df.Node[cause_node[0]])))
    g.add((cause, OWL.sameAs, URIRef(str(node_df.IRI[cause_node[0]])))) 
#     g.add((URIRef(relation + "_cause"), OWL.sameAs, Literal(node_df.Gracious[cause_node[0]])))
    
    #relation
    g.add((relation, URIRef(sbdbel + "relationship"), URIRef(sbdbel + str(selected_row.causes))))
    
    #outcome
    g.add((relation, URIRef(sbdbel + "outcome"), outcome))
    g.add((outcome, DCTERMS.label, Literal(node_df.Node[outcome_node[0]])))
    g.add((outcome, OWL.sameAs, URIRef(str(node_df.IRI[outcome_node[0]]))))
#     g.add((URIRef(relation + "_outcome"), OWL.sameAs, Literal(node_df.Gracious[outcome_node[0]])))
    
g.serialize(format='n3', destination='outp.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [11]:
# <Carbon_Nanoforms_BEL/1> a sbdbel:CausalAssertion ;
#     dc:source <https://h2020-sbd4nano.github.io/sbd-data-landscape/wp1/bel_ralph/> ;
#     cito:cites <https://doi.org/10.3390/coatings11040414> ;
#     sbdbel:hasHypothesis "graphene oxide nanosheets with a higher number of sheets show more toxicity to human keratinocyte HaCaT cells then less number of sheets." ;
#     sbdbel:NP [ rdfs:label "GO sheets" ;
#         sbdbel:hasShapeCategory "platelets" ;
#         sbdbel:hasConstituent "graphene oxide" ; 
#         sbdbel:term <> ; 
#         sbdbel:isCoated <>] ;
#     sbdbel:cause [ rdfs:label "Number of sheets" ; 
#         sbdbel:termCat "PC_ASPECT_RATIO_SHAPE" ; 
#         sbdbel:term "EP_NUMBER_OF_SHEETS"] ;
#     sbdbel:relationship sbdbel:negativelyCorrelates ;
#     sbdbel:outcome [ rdfs:label "Cell viability" ; 
#         sbdbel:termCat "HH_CYTOTOXITIY" ; 
#         sbdbel:term "EP_CELL_VIABILITY"] ;
#     sbdbel:cellType "HaCAT cells" .